In [1]:
import pandas as pd
import psycopg2
#JVcVsBmAki
#JVcVsBmAki
conn = psycopg2.connect(
    dbname="recruiting",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=6543  # local port you forwarded
)
query = """SELECT
  p.id,
  p."fullName",
  STRING_AGG(DISTINCT COALESCE(e."schoolName", '') || ' (' || COALESCE(e.degree, '') || ' in ' || COALESCE(e."fieldOfStudy", '') || ')', '; ') AS education_summary,
  STRING_AGG(DISTINCT COALESCE(pos.title, '') || ' at ' || COALESCE(pos."companyName", '') || ' in ' || COALESCE(pos.location, ''), '; ') AS position_summary,
  p.summary,
  p.skills,
  p.location,
  p.country,
  p.city
FROM person_data p
LEFT JOIN education_data e ON p.username = e.username
LEFT JOIN position_data pos ON p.username = pos.username
GROUP BY p.id, p."fullName", p.summary, p.skills, p.location, p.country, p.city;

"""

df_raw = pd.read_sql_query(query, conn)

# Create the 'text' column by concatenating all fields into a string
df = df_raw.copy()
df["text"] = df.apply(lambda row: f"""
Full Name: {row['fullName']}.
Education: {row['education_summary']}.
Experience: {row['position_summary']}.
Summary: {row['summary']}.
Skills: {row['skills']}.
Location: {row['location']}, {row['city']}, {row['country']}.
""", axis=1)

df = df[["id", "text"]]
df

/var/folders/0_/14m9hb7n2234bn6kbkgxxvbr0000gn/T/ipykernel_31585/528877350.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql_query(query, conn)


,id,text
0,1,\nFull Name: Ayush Jha.\nEducation: Birla Inst...
1,2,\nFull Name: Buket Comert.\nEducation: Boğaziç...
2,3,\nFull Name: Ahmad Sekmani.\nEducation: The Fo...
3,4,\nFull Name: Arpine Lputyan.\nEducation: Frenc...
4,5,\nFull Name: Alper Özer.\nEducation: Izmir Uni...
...,...,...
198,199,\nFull Name: Mat Silva.\nEducation: Cursos PM3...
199,200,\nFull Name: Beatriz Bottino.\nEducation: Univ...
200,201,\nFull Name: Iman Behboudi.\nEducation: comput...
201,202,\nFull Name: DUYGU KÖPÜKLÜ.\nEducation: Anadol...


In [38]:
import pandas as pd
import requests
import dotenv

dotenv.load_dotenv()
# Example dataframe

vacancy_text = """
Название должности: Product Manager
Альтернативные тайтлы: Country Manager, Business Development Manager, Regional Manager
Опыт в годах: 3
Обязательные навыки: Product development, Advanced Spanish, Latam region, Native Russian
Дополнительно важно: важен русскоговорящий к-т
Полезные навыки: New markets, Sales management, market research, budgeting
Домены: must have  iGaming, Gambling
Локации поиска: Кипр, Латвия, Грузия, Мальта, Польша, Сербия, Мексика, Испания, Эстония, Россия, Болгария, Литва, Казахстан, Португалия, Таиланд, Бразилия, Аргентина, Чили, Турция
Гражданство: не Армении, не Грузии

"""

# Convert dataframe to list of dicts
candidates = df.to_dict(orient="records")

# Request payload
payload = {
    "vacancy_text": vacancy_text,
    "candidates": candidates
}

# Send POST request
response = requests.post("http://localhost:8000/match_candidates", json=payload)
print(response.text)


KeyboardInterrupt: 

In [39]:
import json
data = json.loads(response.text)

# The response contains two keys: "custom_id" and "content",
# each mapping indices to the appropriate values.
rows = []
# Loop over the sorted keys (as integers) to maintain order.
for idx in sorted(data["custom_id"], key=lambda x: int(x)):
    custom_id = data["custom_id"][idx]
    # The 'content' is itself a JSON string (structured output)
    content_str = data["content"][idx]
    try:
        # Convert the JSON string to an object (dictionary)
        content_obj = json.loads(content_str)
    except json.JSONDecodeError:
        content_obj = content_str
    rows.append({
        "custom_id": custom_id,
        "content": content_obj
    })

# Create DataFrame:
df_results = pd.DataFrame(rows)
df_results

,custom_id,content
0,1,"{'experience': {'feature': 'Experience', 'scor..."
1,2,"{'experience': {'feature': 'Experience', 'scor..."
2,3,"{'experience': {'feature': 'Experience', 'scor..."
3,4,"{'experience': {'feature': 'Experience', 'scor..."
4,5,"{'experience': {'feature': 'Experience', 'scor..."
...,...,...
198,199,"{'experience': {'feature': 'Experience', 'scor..."
199,200,"{'experience': {'feature': 'Experience', 'scor..."
200,201,"{'experience': {'feature': 'Experience', 'scor..."
201,202,"{'experience': {'feature': 'Experience', 'scor..."


In [40]:
#now flatten the content column {'experience': {'feature': 'Experience in product management', 'score': 3, 'explanation': 'The candidate has 3 years of experience, transitioning from a Project Manager to a Product Manager, which closely aligns with the required experience level for the role.'}, 'skills': {'feature': 'Relevant skills for product management', 'score': 3, 'explanation': 'The candidate possesses product development and management skills but lacks proficiency in advanced Spanish and knowledge of Latin American markets, which are essential requirements.'}, 'domain': {'feature': 'Experience in the iGaming or Gambling industry', 'score': 0, 'explanation': 'The candidate does not mention any experience in the iGaming or Gambling domains, which is a necessary criterion for this role.'}, 'location': {'feature': 'Willingness to relocate from Armenia', 'score': 0, 'explanation': 'The candidate is currently based in Armenia and does not meet the citizenship requirement as they are Armenian.'}, 'overall_summary': {'score': 1, 'summary': 'The candidate demonstrates relevant experience and product management skills, but fails to meet critical requirements including domain experience in iGaming/Gambling, advanced Spanish proficiency, and appropriate citizenship for the role.'}}

# with id

df_results = pd.json_normalize(df_results["content"])
df_results["custom_id"] = df_results.index

df_results

,experience.feature,experience.score,experience.explanation,skills.feature,skills.score,skills.explanation,domain.feature,domain.score,domain.explanation,location.feature,location.score,location.explanation,overall_summary.score,overall_summary.summary,custom_id
0,Experience,10,"Ayush has 7 years of experience, which exceeds...",Mandatory Skills,2,The candidate does not have advanced Spanish s...,Domain Expertise,2,The candidate lacks explicit experience in the...,Location,1,"The candidate is located in Bangkok, Thailand,...",15,While Ayush Jha has extensive experience that ...,0
1,Experience,2,The candidate has several years of experience ...,Skills,-10,"The candidate lacks advanced Spanish, which is...",Domain Expertise,-10,The candidate does not demonstrate experience ...,Location,-10,"The candidate is based in Istanbul, Turkey, wh...",-28,"Overall, the candidate lacks key mandatory ski...",1
2,Experience,5,The candidate has over eight years of experien...,Skills,2,The candidate lacks the mandatory skills in Ad...,Domain Expertise,2,The candidate does not have experience in iGam...,Location,1,"The candidate is located in Istanbul, Turkey, ...",2,The candidate is overqualified in terms of exp...,2
3,Experience,2,The candidate has 3 years of relevant experien...,Skills Alignment,0,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Yerevan, Armenia, ...",0,The candidate is not suitable for the position...,3
4,Experience,2,The candidate has significant experience in pr...,Skills,0,The candidate lacks mandatory skills in produc...,Domain Expertise,0,The candidate does not possess experience in t...,Location,0,"The candidate is located in İzmir, Turkey, whi...",0,The candidate does not meet essential requirem...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Experience,10,"The candidate has 12 years of experience, whic...",Skills,0,While the candidate is trilingual and possesse...,Domain Expertise,0,The candidate does not have specified experien...,Location,0,"The candidate is located in São Paulo, Brazil....",2,The candidate has strong experience and many r...,198
199,Experience,2,Candidate has significant experience but lacks...,Skills and Domain Expertise,0,Candidate does not meet the mandatory skills r...,Domain Expertise,0,Candidate has no experience in iGaming and Gam...,Location,0,"Candidate is located in Brazil, which is not a...",1,"Overall, the candidate is unsuitable due to fa...",199
200,Experience,0,The candidate has substantial experience in pr...,Skills,0,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate's experience does not explicitly...,Location,-50,"The candidate is located in Istanbul, Türkiye,...",-50,"Overall, the candidate falls short due to miss...",200
201,Experience,2,The candidate has several years of experience ...,Skills,1,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate has no experience or expertise i...,Location,0,"The candidate is located in Istanbul, Turkey, ...",1,The candidate has relevant experience but lack...,201


In [41]:
# str id
df["id"] = df.index
df["id"] = df["id"].astype(str)
df

,id,text
0,0,\nFull Name: Ayush Jha.\nEducation: Birla Inst...
1,1,\nFull Name: Buket Comert.\nEducation: Boğaziç...
2,2,\nFull Name: Ahmad Sekmani.\nEducation: The Fo...
3,3,\nFull Name: Arpine Lputyan.\nEducation: Frenc...
4,4,\nFull Name: Alper Özer.\nEducation: Izmir Uni...
...,...,...
198,198,\nFull Name: Mat Silva.\nEducation: Cursos PM3...
199,199,\nFull Name: Beatriz Bottino.\nEducation: Univ...
200,200,\nFull Name: Iman Behboudi.\nEducation: comput...
201,201,\nFull Name: DUYGU KÖPÜKLÜ.\nEducation: Anadol...


In [42]:

df_results["custom_id"] = df_results["custom_id"].astype(str)
#merge with df
df_results = df_results.merge(df, left_on="custom_id", right_on="id", how="left")
df_results = df_results.drop(columns=["custom_id", "id"])
df_results

,experience.feature,experience.score,experience.explanation,skills.feature,skills.score,skills.explanation,domain.feature,domain.score,domain.explanation,location.feature,location.score,location.explanation,overall_summary.score,overall_summary.summary,text
0,Experience,10,"Ayush has 7 years of experience, which exceeds...",Mandatory Skills,2,The candidate does not have advanced Spanish s...,Domain Expertise,2,The candidate lacks explicit experience in the...,Location,1,"The candidate is located in Bangkok, Thailand,...",15,While Ayush Jha has extensive experience that ...,\nFull Name: Ayush Jha.\nEducation: Birla Inst...
1,Experience,2,The candidate has several years of experience ...,Skills,-10,"The candidate lacks advanced Spanish, which is...",Domain Expertise,-10,The candidate does not demonstrate experience ...,Location,-10,"The candidate is based in Istanbul, Turkey, wh...",-28,"Overall, the candidate lacks key mandatory ski...",\nFull Name: Buket Comert.\nEducation: Boğaziç...
2,Experience,5,The candidate has over eight years of experien...,Skills,2,The candidate lacks the mandatory skills in Ad...,Domain Expertise,2,The candidate does not have experience in iGam...,Location,1,"The candidate is located in Istanbul, Turkey, ...",2,The candidate is overqualified in terms of exp...,\nFull Name: Ahmad Sekmani.\nEducation: The Fo...
3,Experience,2,The candidate has 3 years of relevant experien...,Skills Alignment,0,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Yerevan, Armenia, ...",0,The candidate is not suitable for the position...,\nFull Name: Arpine Lputyan.\nEducation: Frenc...
4,Experience,2,The candidate has significant experience in pr...,Skills,0,The candidate lacks mandatory skills in produc...,Domain Expertise,0,The candidate does not possess experience in t...,Location,0,"The candidate is located in İzmir, Turkey, whi...",0,The candidate does not meet essential requirem...,\nFull Name: Alper Özer.\nEducation: Izmir Uni...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Experience,10,"The candidate has 12 years of experience, whic...",Skills,0,While the candidate is trilingual and possesse...,Domain Expertise,0,The candidate does not have specified experien...,Location,0,"The candidate is located in São Paulo, Brazil....",2,The candidate has strong experience and many r...,\nFull Name: Mat Silva.\nEducation: Cursos PM3...
199,Experience,2,Candidate has significant experience but lacks...,Skills and Domain Expertise,0,Candidate does not meet the mandatory skills r...,Domain Expertise,0,Candidate has no experience in iGaming and Gam...,Location,0,"Candidate is located in Brazil, which is not a...",1,"Overall, the candidate is unsuitable due to fa...",\nFull Name: Beatriz Bottino.\nEducation: Univ...
200,Experience,0,The candidate has substantial experience in pr...,Skills,0,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate's experience does not explicitly...,Location,-50,"The candidate is located in Istanbul, Türkiye,...",-50,"Overall, the candidate falls short due to miss...",\nFull Name: Iman Behboudi.\nEducation: comput...
201,Experience,2,The candidate has several years of experience ...,Skills,1,The candidate does not possess advanced Spanis...,Domain Expertise,0,The candidate has no experience or expertise i...,Location,0,"The candidate is located in Istanbul, Turkey, ...",1,The candidate has relevant experience but lack...,\nFull Name: DUYGU KÖPÜKLÜ.\nEducation: Anadol...


In [43]:
df_results.to_csv("results.csv", index=False)